In [1]:
import utils
import emission
import transition

In [2]:
dataset_folder = "data/EN/"
train_data = dataset_folder + "train"
lines = utils.read_file_to_lines(train_data)

In [3]:
emission_data = emission.generate_emission_table(lines)
hashmap = emission_data["x_hashmap"]
word_freq = emission_data["x_word_freq"]
smoothed_hashmap = utils.add_unk(hashmap, word_freq, k=3)
emission_data["x_hashmap"] = smoothed_hashmap

x_vocab = utils.get_emission_vocab(smoothed_hashmap)

In [4]:
transition_pairs = transition.generate_transition_pairs(lines)

In [5]:
y_pairs = transition_pairs["Y_pairs"]
y_vocab = transition_pairs["y_vocab"]
y_freq = transition_pairs["y_freq"]

In [6]:
transition_data = transition.generate_transition_data(y_pairs, y_vocab)

In [7]:
import viterbi

In [8]:
# convert first sentence to test

first_break = lines.index("")
sentence_xy = lines[:first_break]

words, pos_list = [], []
for token in sentence_xy:
    word, pos = token.split(" ")
    words.append(word)
    pos_list.append(pos)

In [9]:
hmm = viterbi.HMM()
hmm.fit_word_tokenizer(x_vocab)
hmm.fit_pos_tokenizer(y_vocab)
hmm.build_transition_weights(y_freq, transition_data)
hmm.build_emission_weights(emission_data)

In [13]:
train_data = dataset_folder + "dev.in"
lines = utils.read_file_to_lines(train_data)

sentences = []

while len(lines) > 1:
    sentence_break = lines.index("")
    sentence_xy = lines[:sentence_break]
    words = [token.strip() for token in sentence_xy]
    sentence = " ".join(words).strip()
    sentences.append(sentence)
    lines = lines[sentence_break+1:]

In [14]:
# only for the progress bar!
try:
    from tqdm import tqdm
    USE_TQDM = True
except Exception as e:
    print(e, "TQDM import error, disable progress bar")

if USE_TQDM:
    sentences_it = tqdm(sentences)
else:
    sentences_it = sentences

  0%|          | 0/1094 [00:00<?, ?it/s]

In [15]:
preds = []

for line in sentences_it:
    pred = hmm.viterbi_predict(line)
    pred = hmm.pos_tokens_to_labels(pred)
    preds.append(pred)
    
assert len(sentences) == len(preds)

100%|██████████| 1094/1094 [00:31<00:00, 34.99it/s]


In [16]:
outfile = dataset_folder + "dev.p3.out"

with open(outfile, "w") as f:
    for sentence, pred in zip(sentences, preds):
        word_array = sentence.split(" ")
        try:
            assert len(word_array) == len(pred)
            for i, word in enumerate(word_array):
                f.write(word + " " + pred[i] +"\n")
        except:
            print(word_array)
            print(pred)
            break
        f.write("\n")

In [17]:
gold_data = dataset_folder + "dev.out"
pred_data = dataset_folder + "dev.p3.out"

data = utils.run_eval(gold_data, pred_data)

print("Entity F:", data["entity_f"])
print("Sentiment F:", data["sentiment_f"])

Entity F: 0.8322
Sentiment F: 0.799
